In [4]:
ctx = SparkSession.builder \
    .master("spark://h:7077") \
    .appName("bosch spark") \
    .getOrCreate()
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"

In [10]:
def load_csv_df(file_name):
    return ctx.read.csv(file_dir + file_name, header=True, inferSchema=True)

In [11]:
n_df = load_csv_df("train_numeric.csv")

In [ ]:
test_df = load_csv_df("tes")